In [ ]:
import cv2
import torch
import easyocr
import pytesseract
import time  # Importar la biblioteca time
import pandas as pd
from pytesseract import Output
from ultralytics import YOLO

# --- Configuración de Rutas de Archivos ---
video_input = "C0142.mp4"
video_output = "resultado.mp4"
csv_output = "resultados.csv"

# --- Configuración de Dispositivo (Apple Silicon/CPU) ---
# Usa 'mps' para Macs con chips M (M1, M2, etc.) y PyTorch compatible
# De lo contrario, usa 'cpu'
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f"Usando dispositivo: {device}")

# --- Inicialización de Modelos y Lectores ---
# EasyOCR: Usará la CPU, ya que MPS no es oficialmente soportado como CUDA
easy_reader = easyocr.Reader(['es'], gpu=False)

# YOLO: Carga los modelos y los mueve al dispositivo determinado ('mps' o 'cpu')
modelo_vehiculos = YOLO("yolo11n.pt").to(device)
modelo_matriculas = YOLO("runs/detect/train3/weights/best.pt").to(device)

# --- Configuración de Video ---
cap = cv2.VideoCapture(video_input)
if not cap.isOpened():
    print(f"Error al abrir el archivo de video: {video_input}")
    exit()

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# El códec 'mp4v' es generalmente compatible, pero podrías necesitar 'avc1' o 'MJPG'
# si 'mp4v' da problemas en macOS.
out = cv2.VideoWriter(video_output, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height)) 

resultados = []
frame_idx = 0

print("Procesando video...")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # --- Medición del tiempo de inferencia (YOLO Vehículos) ---
    start_time = time.time()  # Iniciar cronómetro
    pred = modelo_vehiculos.track(frame, persist=True)[0] 
    end_time = time.time()  # Detener cronómetro
    
    # Calcular el tiempo de inferencia de este frame
    yolo_inference_time = end_time - start_time
    # --- Fin de la medición ---

    for obj in pred.boxes:
        clase_id = int(obj.cls[0])
        clase_nombre = modelo_vehiculos.names[clase_id]

        if clase_nombre not in ['person', 'car']:
            continue

        # Uso de 'round' para asegurar que los límites son enteros antes de map(int, ...)
        x1, y1, x2, y2 = map(int, obj.xyxy[0].round().tolist())
        conf = float(obj.conf[0])
        track_id = int(obj.id[0]) if obj.id is not None and len(obj.id) > 0 else -1 # Manejo más seguro de track_id

        # Inicializar variables para este objeto
        matricula_text_easy = ""
        matricula_text_tess = ""
        easyocr_time = 0.0
        tesseract_time = 0.0

        if clase_nombre == 'car':
            roi = frame[y1:y2, x1:x2]
            if roi.size > 0:
                # YOLO para matrículas dentro del coche (se ejecuta en 'mps' o 'cpu')
                pred_matricula = modelo_matriculas(roi)[0]
                if pred_matricula.boxes:
                    # Uso de 'round' para asegurar que los límites son enteros
                    mx1, my1, mx2, my2 = map(int, pred_matricula.boxes.xyxy[0].round().tolist())
                    roi_matricula = roi[my1:my2, mx1:mx2]

                    if roi_matricula.size > 0:
                        
                        # --- Medición EasyOCR ---
                        start_easy = time.time()
                        res_easy = easy_reader.readtext(roi_matricula)
                        easyocr_time = time.time() - start_easy
                        # --- Fin Medición ---
                        
                        if res_easy:
                            matricula_text_easy = res_easy[0][1]

                        # --- Medición Tesseract OCR ---
                        roi_rgb = cv2.cvtColor(roi_matricula, cv2.COLOR_BGR2RGB)
                        start_tess = time.time()
                        tess_data = pytesseract.image_to_data(roi_rgb, output_type=Output.DICT) 
                        tesseract_time = time.time() - start_tess
                        # --- Fin Medición ---

                        for i, text in enumerate(tess_data['text']):
                            if int(tess_data['conf'][i]) > 0:
                                matricula_text_tess = text
                                break

                    # Dibujar caja matrícula y textos
                    cv2.rectangle(frame, (x1+mx1, y1+my1), (x1+mx2, y1+my2), (255,0,0), 2)
                    cv2.putText(frame, matricula_text_easy, (x1+mx1, y1+my1-10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)
                    cv2.putText(frame, matricula_text_tess, (x1+mx1, y1+my2+15),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)

        # Dibujar caja vehículo/persona
        color = (0,255,0) if clase_nombre=='person' else (0,0,255)
        cv2.rectangle(frame,(x1,y1),(x2,y2),color,2)
        cv2.putText(frame,f"{clase_nombre} ID:{track_id}",(x1,y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX,0.5,color,2)

        resultados.append({
            "frame": frame_idx,
            "tipo": clase_nombre,
            "conf": conf,
            "id": track_id,
            "x1": x1, "y1": y1, "x2": x2, "y2": y2,
            "matricula_easy": matricula_text_easy,
            "matricula_tesseract": matricula_text_tess,
            "yolo_inference_time_s": yolo_inference_time,
            "easyocr_inference_time_s": easyocr_time,
            "tesseract_inference_time_s": tesseract_time
        })

    out.write(frame)
    frame_idx += 1
    
    if frame_idx % 100 == 0:
        print(f"Procesados {frame_idx} frames...")

# --- Finalización ---
cap.release()
out.release()
cv2.destroyAllWindows()

# Guardar CSV
df = pd.DataFrame(resultados)
df.to_csv(csv_output, index=False)
print(f"Finalizado. CSV ({csv_output}) y vídeo ({video_output}) generados.")

# --- Calcular Tiempos Promedio ---
# Imprime un encabezado para la nueva sección de análisis
print("\n--- Tiempos de Inferencia Promedio (excluyendo 0.0s) ---")

# Para YOLO, filtramos los tiempos mayores a 0 (aunque casi siempre será > 0)
yolo_times = df[df['yolo_inference_time_s'] > 0]['yolo_inference_time_s']
if not yolo_times.empty:
    # .mean() calcula la media, :.4f formatea a 4 decimales
    print(f"Promedio YOLO (Vehículos):   {yolo_times.mean():.4f} segundos")
else:
    print("YOLO (Vehículos): Sin datos válidos.")

# Para EasyOCR, filtramos solo donde realmente se ejecutó (tiempo > 0)
easyocr_times = df[df['easyocr_inference_time_s'] > 0]['easyocr_inference_time_s']
if not easyocr_times.empty:
    print(f"Promedio EasyOCR (Matrículas): {easyocr_times.mean():.4f} segundos")
else:
    print("EasyOCR (Matrículas): No se ejecutó o no hay datos.")

# Para Tesseract, filtramos solo donde realmente se ejecutó (tiempo > 0)
tesseract_times = df[df['tesseract_inference_time_s'] > 0]['tesseract_inference_time_s']
if not tesseract_times.empty:
    print(f"Promedio Tesseract (Matrículas): {tesseract_times.mean():.4f} segundos")
else:
    print("Tesseract (Matrículas): No se ejecutó o no hay datos.")

Using CPU. Note: This module is much faster with a GPU.


Usando dispositivo: mps
Procesando video...

0: 384x640 2 persons, 11 cars, 33.7ms
Speed: 1.4ms preprocess, 33.7ms inference, 6.6ms postprocess per image at shape (1, 3, 384, 640)

0: 256x416 1 matricula, 26.2ms
Speed: 0.6ms preprocess, 26.2ms inference, 7.4ms postprocess per image at shape (1, 3, 256, 416)

0: 288x416 1 matricula, 43.4ms
Speed: 1.7ms preprocess, 43.4ms inference, 7.2ms postprocess per image at shape (1, 3, 288, 416)

0: 352x416 (no detections), 43.4ms
Speed: 1.4ms preprocess, 43.4ms inference, 2.9ms postprocess per image at shape (1, 3, 352, 416)

0: 416x416 (no detections), 27.8ms
Speed: 1.1ms preprocess, 27.8ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 416)

0: 288x416 (no detections), 24.8ms
Speed: 1.0ms preprocess, 24.8ms inference, 1.3ms postprocess per image at shape (1, 3, 288, 416)

0: 288x416 (no detections), 9.9ms
Speed: 1.1ms preprocess, 9.9ms inference, 2.3ms postprocess per image at shape (1, 3, 288, 416)

0: 256x416 (no detections), 35.

/Users/davidmiranda/.local/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


0: 320x416 1 matricula, 45.3ms
Speed: 1.0ms preprocess, 45.3ms inference, 9.5ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 13.1ms
Speed: 1.8ms preprocess, 13.1ms inference, 3.4ms postprocess per image at shape (1, 3, 320, 416)

0: 416x384 (no detections), 27.4ms
Speed: 0.9ms preprocess, 27.4ms inference, 1.6ms postprocess per image at shape (1, 3, 416, 384)

0: 288x416 (no detections), 36.0ms
Speed: 0.8ms preprocess, 36.0ms inference, 1.2ms postprocess per image at shape (1, 3, 288, 416)

0: 384x640 2 persons, 10 cars, 1 bench, 28.4ms
Speed: 2.1ms preprocess, 28.4ms inference, 6.5ms postprocess per image at shape (1, 3, 384, 640)

0: 320x416 (no detections), 27.9ms
Speed: 0.8ms preprocess, 27.9ms inference, 2.2ms postprocess per image at shape (1, 3, 320, 416)

0: 288x416 1 matricula, 22.3ms
Speed: 1.4ms preprocess, 22.3ms inference, 4.7ms postprocess per image at shape (1, 3, 288, 416)

0: 416x416 (no detections), 33.1ms
Speed: 1.9ms preprocess, 33.1m

/Users/davidmiranda/.local/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


...

0: 288x416 (no detections), 23.0ms
Speed: 1.2ms preprocess, 23.0ms inference, 1.1ms postprocess per image at shape (1, 3, 288, 416)

0: 288x416 (no detections), 9.4ms
Speed: 1.3ms preprocess, 9.4ms inference, 0.9ms postprocess per image at shape (1, 3, 288, 416)

0: 416x352 (no detections), 29.3ms
Speed: 0.8ms preprocess, 29.3ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 352)

0: 288x416 (no detections), 26.6ms
Speed: 0.9ms preprocess, 26.6ms inference, 1.3ms postprocess per image at shape (1, 3, 288, 416)

0: 288x416 (no detections), 8.5ms
Speed: 1.0ms preprocess, 8.5ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 416)

0: 352x416 (no detections), 32.5ms
Speed: 1.0ms preprocess, 32.5ms inference, 1.4ms postprocess per image at shape (1, 3, 352, 416)

0: 384x640 1 person, 10 cars, 27.0ms
Speed: 1.7ms preprocess, 27.0ms inference, 6.7ms postprocess per image at shape (1, 3, 384, 640)

0: 288x416 1 matricula, 24.5ms
Speed: 1.0ms preprocess, 24.5ms inferenc

/Users/davidmiranda/.local/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


0: 288x416 (no detections), 29.2ms
Speed: 1.3ms preprocess, 29.2ms inference, 1.1ms postprocess per image at shape (1, 3, 288, 416)

0: 288x416 (no detections), 9.4ms
Speed: 1.0ms preprocess, 9.4ms inference, 2.5ms postprocess per image at shape (1, 3, 288, 416)

0: 416x352 (no detections), 33.5ms
Speed: 1.1ms preprocess, 33.5ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 352)

0: 288x416 (no detections), 21.4ms
Speed: 1.0ms preprocess, 21.4ms inference, 1.1ms postprocess per image at shape (1, 3, 288, 416)

0: 288x416 (no detections), 9.3ms
Speed: 0.8ms preprocess, 9.3ms inference, 2.4ms postprocess per image at shape (1, 3, 288, 416)

0: 352x416 (no detections), 27.7ms
Speed: 1.0ms preprocess, 27.7ms inference, 1.4ms postprocess per image at shape (1, 3, 352, 416)

0: 384x640 1 person, 10 cars, 27.2ms
Speed: 1.6ms preprocess, 27.2ms inference, 4.5ms postprocess per image at shape (1, 3, 384, 640)

0: 288x416 1 matricula, 25.4ms
Speed: 1.2ms preprocess, 25.4ms inferenc

/Users/davidmiranda/.local/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


0: 288x416 (no detections), 25.5ms
Speed: 1.1ms preprocess, 25.5ms inference, 1.1ms postprocess per image at shape (1, 3, 288, 416)

0: 288x416 (no detections), 8.9ms
Speed: 1.0ms preprocess, 8.9ms inference, 0.9ms postprocess per image at shape (1, 3, 288, 416)

0: 416x352 (no detections), 30.2ms
Speed: 0.8ms preprocess, 30.2ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 352)

0: 288x416 (no detections), 25.6ms
Speed: 0.6ms preprocess, 25.6ms inference, 1.2ms postprocess per image at shape (1, 3, 288, 416)

0: 288x416 (no detections), 8.5ms
Speed: 0.9ms preprocess, 8.5ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 416)

0: 352x416 (no detections), 27.2ms
Speed: 1.2ms preprocess, 27.2ms inference, 1.4ms postprocess per image at shape (1, 3, 352, 416)

0: 384x640 1 person, 10 cars, 27.1ms
Speed: 1.6ms preprocess, 27.1ms inference, 7.1ms postprocess per image at shape (1, 3, 384, 640)

0: 288x416 2 matriculas, 21.4ms
Speed: 1.2ms preprocess, 21.4ms inferen